In [1]:
import pandas as pd
import rapidfuzz
# import fuzzy_matching

## fuzzy

In [2]:
import rapidfuzz
import pandas as pd
# need to run nltk.download() when using for the first time
# this opens a window, in the download path just make it C:/(nltk_data) or whatever this is called
# this makes it so you dont have to set an environmental variable
import nltk 

mapi = pd.read_csv(r"C:\Users\Jakub\Documents\zazu\openai-quickstart-python\timings\mapi_list_sn_ln.csv")
mapi.iloc[3752][0] = ""


# function for sorting
def key(tup):
    return tup[1]

def convertTuple(tup):
    # initialize an empty string
    str = ''
    for item in tup:
        str = str + " " + item
    return str

# clean the answers before trying to match
def answer_cleaning1(my_list):
    templist1 = []
    for answer in my_list:
        if 'None found' not in answer:
            templist1.append(answer)
    return templist1


def answer_cleaning(my_list):
    temp_list1 = []
    temp_list2 = []
    temp_list3 = []
    temp_list4 = []
    for x in my_list:
        if not 'None found' in x and len(x) < 1001:
            # if not x == 'No validated clinical questionnaires were used in this text.':
            temp_list1.append(x)
    for y in temp_list1:
        split = y.split('\n')
        for s in split:
            temp_list2.append(s)
    for z in temp_list2:
        if not 'not a questionnaire' in z:
            temp_list3.append(z)
    for string in temp_list3:
        tokens = string.split()
        # word 'questionnaire' skews answers so remove it
        skews = ('life','Life','of','questionnaire','questionnaires','Questionnaire','Questionnaires', '-')
        tokens = [token if token not in skews else '' for token in tokens]
        string = ' '.join(tokens)
        string = string.replace(',','').strip()
        temp_list4.append(string)
    # print(temp_list4)
    return temp_list4

# my version of the rapidfuzz.process.extract_iter() which allows for change of weights of the processor
# compares the string with all the long names in the csv above
def weighted_iter_long(string, weight_cut_off=0.8):
    matches = []
    choices = mapi.long_name

    for choice in choices:
        score = rapidfuzz.distance.Levenshtein.normalized_similarity(string, choice, processor=rapidfuzz.utils.default_process, weights=(1,1,1))
        # remove incomplete answers where '...' occurs
        if score >= weight_cut_off and "..." not in choice:
            matches.append((choice, score))
    
    # sort the list in descending order
    matches = sorted(matches, key=key, reverse=True)
    # print('matches', matches)
    
    return matches

# same as above but for the short names
# splits the string to compare the individual tokens to the short names
def weighted_iter_short(string, weight_cut_off=0.8):
    matches = []
    tokens = string.split()
    nltk_tokens = nltk.word_tokenize(string)
    trigrams = list(nltk.ngrams(nltk_tokens,3))
    trigrams = [convertTuple(tup).strip() for tup in trigrams]
    
    bigrams = list(nltk.bigrams(nltk_tokens))
    bigrams = [convertTuple(tup).strip() for tup in bigrams]
    # print('bigrams',bigrams)
    tokens = [token if token != 'questionnaire' else '' for token in tokens]
    choices = mapi.short_name
    for token in tokens:
        for choice in choices:
            score = rapidfuzz.distance.Levenshtein.normalized_similarity(token, choice, processor=rapidfuzz.utils.default_process, weights=(1,1,1))
            if score >= weight_cut_off and "..." not in choice:
                matches.append((choice, score))
    for token in bigrams:
        for choice in choices:
            score = rapidfuzz.distance.Levenshtein.normalized_similarity(token, choice, processor=rapidfuzz.utils.default_process, weights=(1,1,1))
            if score >= weight_cut_off and "..." not in choice:
                matches.append((choice, score))
    for token in trigrams:
        for choice in choices:
            score = rapidfuzz.distance.Levenshtein.normalized_similarity(token, choice, processor=rapidfuzz.utils.default_process, weights=(1,1,1))
            if score >= weight_cut_off and "..." not in choice:
                matches.append((choice, score))    
        
    matches = sorted(matches, key=key, reverse=True)
    # print('matches', matches)
    
    return matches


# join three algorithms to match long name
# all normalized scores are added together to find the closest match
# cutoffs can be changed to increase or the decrease number of potential answers but only the top answer is outputted 
def long_compound(string, weight_cut_off=0.8):

    weighted = weighted_iter_long(string, weight_cut_off=0.8)
    ratio = []
    for answer in weighted:
        y = rapidfuzz.fuzz.partial_ratio(string, answer[0],processor=rapidfuzz.utils.default_process)/100
        score = answer[1] + y
        ratio.append((answer[0], score))
    actual = []
    for answer in ratio:
        x = rapidfuzz.distance.Levenshtein.normalized_similarity(string, answer[0],processor=rapidfuzz.utils.default_process, weights=(1,0.999999,1))
        score = answer[1] + x
        actual.append((answer[0], score))
    
    actual = sorted(actual, key=key, reverse=True)
    # print('long', actual, string)
    cutoff = 2
    if len(actual) > 1:
        if actual[1][1] >= cutoff:
            actual = actual[0]
        else:
            actual = ()
    elif len(actual) == 1:
        if actual[0][1] >= cutoff:
            actual = actual[0]
        else:
            actual = ()
    else:
        actual = ()
    # print(actual,string)
    return actual

# same as above but for short names
def short_compound(string,weight_cut_off=0.8):
    tokens = string.split()
    weighted = weighted_iter_short(string,weight_cut_off)
    print('w', weighted)
    ratio = []
    for answer in weighted:
        for token in tokens:
            y = rapidfuzz.fuzz.partial_ratio(token, answer[0],processor=rapidfuzz.utils.default_process)/100
            score = answer[1] + y
            ratio.append((answer[0], score))
    print('ratio',ratio)
    actual = []
    for answer in ratio:
        for token in tokens:
            x = rapidfuzz.distance.Levenshtein.normalized_similarity(string, answer[0], weights=(1,0.999999,1),processor=rapidfuzz.utils.default_process)
            score = answer[1] + x
            actual.append((answer[0], score))
    actual = list(filter(None, actual))
    actual = list(dict.fromkeys(actual))
    actual = sorted(actual, key=key, reverse=True)
    print('actual',actual)
    
    cutoff = 2.4
    if len(actual) > 1:
        print(actual[0])
        if actual[0][1] >= cutoff:
            actual = actual[0]
        else:
            actual = ()
    elif len(actual) == 1:
        if actual[0][1] >= cutoff:
            actual = actual[0]
        else:
            actual = ()
    else:
        actual = ()
    print('short', actual,string)
    return actual


# join the short + long name functions, taking the best score 
# s stands for short name and l for long name
def joint(string, weight_cut_off=0.8):
    short = short_compound(string, weight_cut_off=0.8)
    long  = long_compound(string, weight_cut_off=0.8)
    print(string, short , long)
    best = (long, 'l')
    if short and not long:
        best = (short,'s')
    if short and long:
        if short[1] > long[1]:
            best = (short,'s')
    if best[0]:
        best = best[0][0], best[1]
    return best

# cleans the questionnaires and matches long and short names, outputting a dictionary
def questionnaire_output(my_list):
    new = list(filter(None, my_list))
    almost = {}
    short = []
    long = []
    for q in new:
        if q[1] == 's':
            short.append(q[0])
        else:
            long.append(q[0])
    short = list(filter(None, short))
    long = list(filter(None, long))
    for q in short:
        idx = mapi.index[mapi['short_name'] == q]
        if mapi.iloc[idx[0]][0] not in almost:
            almost[mapi.iloc[idx[0]][0]] = mapi.iloc[idx[0]][1]

    for q in long:
        idx = mapi.index[mapi['long_name'] == q]
        if mapi.iloc[idx[0]][0] not in almost:
            almost[mapi.iloc[idx[0]][0]] = mapi.iloc[idx[0]][1]


    return almost

def fuzzy_matching(answers, weight_cut_off=0.8):
    cleaned_answers1 = answer_cleaning1(answers)
    cleaned_answers = answer_cleaning(cleaned_answers1)
    cleaned_answers = answer_cleaning(cleaned_answers)
    questionnaires = []
    for answer in cleaned_answers:
        questionnaires.append(joint(answer, weight_cut_off=0.8))
    return questionnaire_output(questionnaires)

## 32

In [3]:
mapi = pd.read_csv("mapi_list_sn_ln.csv")
mapi.iloc[3752][0] = ""

In [65]:
table_32 = pd.read_csv('trial_32_page_29.csv', encoding='unicode_escape')

In [16]:
# df.columns = df.iloc[1]

In [66]:
table_32

,Unnamed: 0,Unnamed: 1,Screening,Unnamed: 3,Placebo-Controlled,Phase,Unnamed: 6,Unnamed: 7,Apremilast,Extension,Phaseb,Post-Treatment Observational Follow-upd
0,1,Visit Number,1,Baseline 2,3,4,5,6,7,8,9/ET°,NaN
1,2,Week,-35 to 0 days,0 (Day 1),2 ( 3 days),4 ( 3 days),16 ( 3 days),20 (= 4 days),32 ( 4 days),44 ( 4 days),52 (= 4 days),56 weeks (or 4 weeks after study discontinuation)
2,3,Informed consente,SELECTED X,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,-
3,4,Inclusion/Exclusion criteria,SELECTED X,SELECTED X,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,-
4,5,Demographics,SELECTED X,SELECTED -,SELECTED -,SELECTED -,SELECTED -,SELECTED -,SELECTED -,NOT_SELECTED -,NOT_SELECTED -,-
5,6,Medical history,SELECTED X,SELECTED -,SELECTED -,SELECTED -,SELECTED -,SELECTED -,SELECTED -,SELECTED -,NOT_SELECTED -,-
6,7,Prior / concomitant medications or therapies,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,NOT_SELECTED -
7,8,Clinical and Laboratory,Assessments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_SELECTED
8,9,Adverse events,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X,SELECTED X
9,10,Pregnancy test and contraception education,SELECTED X,SELECTED X,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,NOT_SELECTED -,SELECTED X,NOT_SELECTED -


In [6]:
for name in df['Unnamed: 1']:
    print(name)

Visit Number
Week
Informed consente
Inclusion/Exclusion criteria
Demographics
Medical history
Prior / concomitant medications or therapies
Clinical and Laboratory
Adverse events
Pregnancy test and contraception education
Vital signs
Height
Weight
Waist Circumference
Body Mass Index
Physical Examination
Clinical laboratory evaluations-h,
Week
Health-related Quality of
DLQI
sPGA of Visible Location
ScPGAk
Nail Assessment
NAPSI
Modified sPGA-Gk
PPPGAK
Itch NRS
Skin Discomfort/Pain VAS
PASI
BSA
EQ-5D
PNQ
PBQ
WPAI: PSO
Photography"
Dispense IP
Return and Count IP Tablets


isolate actual questionnaires

In [ ]:
df.iloc[5]

In [28]:
drop = ['Body Surface Area (BSA)','Patient Benefit Index (PBI)','Treatment-emergent Adverse events','Clinically significant changes in body weight, waist circumference, vital signs, and/or laboratory findings','Body Mass Index']

In [29]:
df1 = df[df['Unnamed: 1'].isin(drop) == False]

In [38]:
weighted_iter_short('PASI')

[('PASI', 1.0), ('PASI', 1.0), ('PAMSI', 0.8), ('PANSI', 0.8)]

In [55]:
short_compound('PASI')

w [('PASI', 1.0), ('PASI', 1.0), ('PAMSI', 0.8), ('PANSI', 0.8)]
ratio [('PASI', 2.0), ('PASI', 2.0), ('PAMSI', 1.55), ('PANSI', 1.55)]
actual [('PASI', 3.0), ('PAMSI', 2.35), ('PANSI', 2.35)]
('PASI', 3.0)
short ('PASI', 3.0) PASI


('PASI', 3.0)

In [56]:
fuzzy_matching(['PASI'])

w [('PASI', 1.0), ('PASI', 1.0), ('PAMSI', 0.8), ('PANSI', 0.8)]
ratio [('PASI', 2.0), ('PASI', 2.0), ('PAMSI', 1.55), ('PANSI', 1.55)]
actual [('PASI', 3.0), ('PAMSI', 2.35), ('PANSI', 2.35)]
('PASI', 3.0)
short ('PASI', 3.0) PASI
PASI ('PASI', 3.0) ()


{'PASI': 'Patient-Specific Index'}

In [32]:
names = []
for name in df1['Unnamed: 1']:
    names.append(name)
que = fuzzy_matching(names)
print(que)

Visit Number () ()
Week () ()
Informed consente () ()
Inclusion/Exclusion criteria () ()
Demographics () ()
Medical history () ()
Prior / concomitant medications or therapies () ()
Clinical and Laboratory () ()
Adverse events () ()
Pregnancy test and contraception education () ()
Vital signs () ()
Height () ()
Weight () ()
Waist Circumference () ()
Physical Examination () ()
Clinical laboratory evaluations-h () ()
Week () ()
Health-related Quality () ()
DLQI ('DLQI', 3.0) ()
sPGA Visible Location () ()
ScPGAk () ()
Nail Assessment () ()
NAPSI ('NAPSI', 3.0) ()
Modified sPGA-Gk () ()
PPPGAK () ()
Itch NRS () ()
Skin Discomfort/Pain VAS () ()
PASI () ()
BSA ('BSA', 3.0) ()
EQ-5D ('EQ-5D', 3.0) ()
PNQ ('PNQ', 3.0) ()
PBQ ('PBQ', 3.0) ()
WPAI: PSO ('WPAI:PSO', 2.8) ()
Photography" () ()
Dispense IP () ()
Return and Count IP Tablets () ()
{'DLQI': 'Dermatology Life Quality Index', 'NAPSI': 'Nail Psoriasis Severity Index', 'BSA': 'Body Surface Area', 'EQ-5D': 'European Quality of Life 5-Dime

In [33]:
que

{'DLQI': 'Dermatology Life Quality Index',
 'NAPSI': 'Nail Psoriasis Severity Index',
 'BSA': 'Body Surface Area',
 'EQ-5D': 'European Quality of Life 5-Dimension Questionnaire',
 'PNQ': 'Patient Neurotoxicity Questionnaire',
 'PBQ': 'Patient Benefit Questionnaire',
 'WPAI:PSO': 'Work Productivity and Activity Impairment Questionnaire: Psoriasis'}

In [10]:
df1[['Unnamed: 1', 'Timeframe']]

KeyError: "['Timeframe'] not in index"

In [77]:
df1

,Unnamed: 0,Endpoint,Name,Description,Timeframe
0,1,Primary,Dermatology Life Quality Index (DLQI),Proportion of subjects who achieve a > 4-point...,Weeks 16
1,2,Secondary,DLQI,Proportion of subjects who achieve a > 4-point...,"Weeks 32, 52"
2,3,NaN,DLQI,Mean change from baseline,"Weeks 16, 32, 52"
3,4,NaN,Itch Numeric Rating Scale (NRS),Mean change from baseline in Itch NRS score,"Weeks 16,32,52"
4,5,NaN,Skin Discomfort/Pain Visual Analog Scale (VAS),Mean change from baseline in skin discomfort/p...,"Weeks 16,32,52"
7,8,NaN,Psoriasis Area Severity Index (PASI),Proportion of subjects who achieve PASI < 3,"Weeks 16,32,52"
8,9,NaN,European Quality of Life 5- Dimension (EQ-5D),Mean percent change from baseline in EQ-5D score,"Weeks 16, 52"
9,10,NaN,Work Productivity and Activity Impairment Ques...,Mean change in WPAI domain scores,"Weeks 16, 52"
12,13,Exploratory,Static Physician Global Assessment (sPGA) of V...,Proportion of subjects who achieve sPGA score ...,"Weeks 16, 32, 52"
13,14,NaN,Scalp Physician Global Assessment (ScPGA),Proportion of subjects who achieve ScPGA score...,"Weeks 16, 32, 52"


In [81]:
df1.reset_index()

C:\Users\Jakub\AppData\Local\Temp\ipykernel_17992\2337605965.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  df1.set_index('index',[range(len(df1['Name']))])


KeyError: "None of ['index'] are in the columns"

In [75]:
df1

<bound method DataFrame.reset_index of     Unnamed: 0     Endpoint  \
0            1      Primary   
1            2    Secondary   
2            3          NaN   
3            4          NaN   
4            5          NaN   
7            8          NaN   
8            9          NaN   
9           10          NaN   
12          13  Exploratory   
13          14          NaN   
14          15          NaN   
15          16          NaN   
16          17          NaN   

                                                 Name  \
0               Dermatology Life Quality Index (DLQI)   
1                                                DLQI   
2                                                DLQI   
3                     Itch Numeric Rating Scale (NRS)   
4      Skin Discomfort/Pain Visual Analog Scale (VAS)   
7                Psoriasis Area Severity Index (PASI)   
8       European Quality of Life 5- Dimension (EQ-5D)   
9   Work Productivity and Activity Impairment Ques...   
12  Static Ph

In [67]:
df1['Name'][5]

KeyError: 5

In [61]:
times = {}
for x in range(len(df1['Name'])):
    times[df1['Name'][x]] = df1['Timeframe'][x]

KeyError: 5

## 112

In [8]:
df = pd.read_csv('table_112.csv', encoding='unicode_escape')

In [9]:
df

,Unnamed: 0,Name,Screening,,Treatment,phase,End of Study,.1,Long-term,follow-uph,.2,Record Review
0,1,,Day -30 to -1,Day 1 (+7 days),Day 30 (+7 days),Day 60 (+7 days),Day 91 (+7 days),Day 180 (+30 days),Day 365 (+30 days),Day 545 (+30 days),Day 730 (+30 days),"Year 3, 4, 5 (+90 days)"
1,2,Informed Consent,SELECTED x,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED
2,3,History and Physicala,SELECTED x,SELECTED x,SELECTED x,SELECTED x,SELECTED x,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED
3,4,Adverse Event Assessment,NOT_SELECTED,SELECTED x,SELECTED x,SELECTED x,SELECTED x,SELECTED x,SELECTED x,SELECTED x,SELECTED x,NOT_SELECTED
4,5,Study drug dispensation,NOT_SELECTED,SELECTED x,SELECTED x,SELECTED x,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED
5,6,Unused study drug collection,NOT_SELECTED,NOT_SELECTED,SELECTED x,SELECTED x,SELECTED x,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED
6,7,CBCb,SELECTED x,SELECTED x,SELECTED x,SELECTED x,SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED
7,8,CMPb,SELECTED x,SELECTED x,SELECTED,SELECTED x,SELECTED x,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED
8,9,EKG,SELECTED x,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED,NOT_SELECTED
9,10,Serum PSA,SELECTED x,SELECTED,NOT_SELECTED,NOT_SELECTED,SELECTED x,SELECTED x,SELECTED x,SELECTED,SELECTED x,NOT_SELECTED


In [10]:
for name in df['Name']:
    print(name)

 
Informed Consent
History and Physicala
Adverse Event Assessment
Study drug dispensation
Unused study drug collection
CBCb
CMPb
EKG
Serum PSA
Testosterone
LH plasma levelb
TSHC
Exploratory biomarkersd
Quality of life questionnaires
Systematic 12-core biopsy
Site directed biopsy (3-cores per site)
Medical Record Review


In [12]:
names = []
for name in df['Name']:
    names.append(name)
que = fuzzy_matching(names)
print(que)

w []
ratio []
actual []
short () 
 () ()
w []
ratio []
actual []
short () Informed Consent
Informed Consent () ()
w []
ratio []
actual []
short () History and Physicala
History and Physicala () ()
w []
ratio []
actual []
short () Adverse Event Assessment
Adverse Event Assessment () ()
w []
ratio []
actual []
short () Study drug dispensation
Study drug dispensation () ()
w []
ratio []
actual []
short () Unused study drug collection
Unused study drug collection () ()
w []
ratio []
actual []
short () CBCb
CBCb () ()
w []
ratio []
actual []
short () CMPb
CMPb () ()
w []
ratio []
actual []
short () EKG
EKG () ()
w [('PSA', 1.0)]
ratio [('PSA', 1.5), ('PSA', 2.0)]
actual [('PSA', 3.0), ('PSA', 2.5)]
('PSA', 3.0)
short ('PSA', 3.0) Serum PSA
Serum PSA ('PSA', 3.0) ()
w []
ratio []
actual []
short () Testosterone
Testosterone () ()
w []
ratio []
actual []
short () LH plasma levelb
LH plasma levelb () ()
w []
ratio []
actual []
short () TSHC
TSHC () ()
w []
ratio []
actual []
short () Explorato